In [24]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import string
import xgboost as xgb

In [2]:
tweets = pd.read_csv("train.csv", usecols=['id','text', 'target'])
test = pd.read_csv("test.csv")

In [3]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7434 non-null   int64 
 1   text    7434 non-null   object
 2   target  7434 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 232.3+ KB


In [4]:
import io
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mausa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def remove_stopword(text):
    new_text = []
    for e in text:
        if e not in stopwords and e.isalpha():
            new_text.append(e)
    text = new_text
    return " ".join(new_text)

def stemm(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

def contains_punctuation(text):
    punctuation = set(string.punctuation)
    for character in text:
        if character in punctuation:
            return True
    return False

def amount_of_punctuation(text):
    punctuation = set(string.punctuation)
    amount = 0
    for character in text:
        if character in punctuation: amount += 1
    return amount

In [6]:
x_train, x_test, y_train, y_test = \
train_test_split(tweets['text'], tweets['target'], test_size = 0.25, random_state = 123)

### Fichur Inginierin


In [7]:
tweets_metrics = tweets[['id','text','target']]
tweets_metrics.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
#Agrego longitud de tweet en caracteres
tweets_metrics['length'] = tweets_metrics['text'].apply(lambda x: len(x))
tweets_metrics.head()

,id,text,target,length
0,1,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,Forest fire near La Ronge Sask. Canada,1,38
2,5,All residents asked to 'shelter in place' are ...,1,133
3,6,"13,000 people receive #wildfires evacuation or...",1,65
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88


In [9]:
tweets_metrics['avg_word_length'] = tweets_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000


In [10]:
tweets_metrics['amount_of_words'] = tweets_metrics['text'].str.split().transform(lambda x: len(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16


In [11]:
unique_words_by_tweet = tweets_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
tweets_metrics['amount_of_unique_words'] = unique_words_by_tweet
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
def return_sia_compound_values(text):
    return sia.polarity_scores(text)['compound']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mausa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
tweets_metrics['sentiment'] = tweets_metrics['text'].apply(lambda x: return_sia_compound_values(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000


In [14]:
tweets_metrics['stopwords_count'] = tweets_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7


In [15]:
tweets_metrics['punctuation_count'] = tweets_metrics['text'].apply(lambda x: amount_of_punctuation(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6,1
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0,1
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11,3
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1,2
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7,2


In [16]:
#tweets_with_mentions = tweets.loc[tweets['text'].str.contains('@'), ['text', 'target']]
mentions = tweets_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()

In [17]:
tweets_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6,1,0
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0,1,0
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11,3,0
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1,2,0
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7,2,0


In [18]:
hashtags = tweets_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
tweets_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count,hashtags_count
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6,1,0,1
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0,1,0,0
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11,3,0,0
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1,2,0,1
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7,2,0,2


In [19]:
tweets_metrics['text_without_stopwords'] = tweets_metrics['text'].str.split()
tweets_metrics['text_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(remove_stopword)
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count,hashtags_count,text_without_stopwords
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6,1,0,1,Our Deeds Reason May ALLAH Forgive us
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0,1,0,0,Forest fire near La Ronge Canada
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11,3,0,0,All residents asked notified No evacuation she...
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1,2,0,1,people receive evacuation orders California
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7,2,0,2,Just got sent photo Ruby smoke pours school


In [20]:
tweets_metrics['longest_word_length_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
tweets_metrics.head()

,id,text,target,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count,hashtags_count,text_without_stopwords,longest_word_length_without_stopwords
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,4.384615,13,13,0.2732,6,1,0,1,Our Deeds Reason May ALLAH Forgive us,7
1,4,Forest fire near La Ronge Sask. Canada,1,38,4.571429,7,7,-0.3400,0,1,0,0,Forest fire near La Ronge Canada,6
2,5,All residents asked to 'shelter in place' are ...,1,133,5.090909,22,20,-0.2960,11,3,0,0,All residents asked notified No evacuation she...,10
3,6,"13,000 people receive #wildfires evacuation or...",1,65,7.125000,8,8,0.0000,1,2,0,1,people receive evacuation orders California,10
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,4.500000,16,15,0.0000,7,2,0,2,Just got sent photo Ruby smoke pours school,6


In [57]:
x_train, x_test, y_train, y_test = \
train_test_split(tweets_metrics[['length','avg_word_length','amount_of_words','amount_of_unique_words','sentiment','stopwords_count','punctuation_count','mentions_count','hashtags_count','longest_word_length_without_stopwords']], tweets_metrics['target'], test_size = 0.25, random_state = 123)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [58]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic',
                colsample_bytree = 0.35, learning_rate = 0.3,
                max_depth = 10, alpha = 10, n_estimators = 100)

In [59]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.35, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='binary:logistic', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [60]:
preds = xg_reg.predict(x_test)
print(preds)

[0.5163653  0.30542597 0.7917713  ... 0.22494091 0.5089964  0.93277615]


In [86]:
for i in range (0, len(preds)):
    if i <= 0.5:       
        preds[i] = 1 
    else:  
        preds[i]=0

In [87]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(preds,y_test)
print(accuracy)

0.5669714900484131


In [88]:
vectorizer = TfidfVectorizer()
tfidf_lgbm = vectorizer.fit_transform(tweets_metrics.loc[:, 'text'])
array = tfidf_lgbm.todense()


In [90]:
df = pd.DataFrame(array)
df['output'] = tweets['target']
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21520,21521,21522,21523,21524,21525,21526,21527,21528,output
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.416232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [91]:
basic_features = tweets_metrics[['length','avg_word_length','amount_of_words','amount_of_unique_words','sentiment','stopwords_count','punctuation_count','mentions_count','hashtags_count','longest_word_length_without_stopwords']]
basic_features.head()

,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count,hashtags_count,longest_word_length_without_stopwords
0,69,4.384615,13,13,0.2732,6,1,0,1,7
1,38,4.571429,7,7,-0.3400,0,1,0,0,6
2,133,5.090909,22,20,-0.2960,11,3,0,0,10
3,65,7.125000,8,8,0.0000,1,2,0,1,10
4,88,4.500000,16,15,0.0000,7,2,0,2,6


In [97]:
tfidf_features = df.merge(basic_features, left_index = True, right_index = True)
tfidf_features

,0,1,2,3,4,5,6,7,8,9,...,length,avg_word_length,amount_of_words,amount_of_unique_words,sentiment,stopwords_count,punctuation_count,mentions_count,hashtags_count,longest_word_length_without_stopwords
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69,4.384615,13,13,0.2732,6,1,0,1,7
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38,4.571429,7,7,-0.3400,0,1,0,0,6
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,133,5.090909,22,20,-0.2960,11,3,0,0,10
3,0.0,0.416232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,65,7.125000,8,8,0.0000,1,2,0,1,10
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,88,4.500000,16,15,0.0000,7,2,0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7429,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,136,7.058824,17,17,-0.7579,2,15,0,1,10
7430,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,105,6.571429,14,14,-0.7579,2,9,0,1,10
7431,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134,5.428571,21,21,0.9164,5,9,0,0,8
7432,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,139,4.384615,26,21,0.4118,9,7,0,0,7


In [98]:
features = tfidf_features.columns.drop('output')
x = tfidf_features.loc[:, features].values
y = tfidf_features.loc[:, 'output'].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 123)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [107]:
xg_reg = xgb.XGBRegressor(objective ='reg:squaredlogerror',
                colsample_bytree = 0.35, learning_rate = 0.3,
                max_depth = 10, alpha = 10, n_estimators = 100)

In [108]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.35, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squaredlogerror', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [109]:
preds = xg_reg.predict(x_test)
print(preds)

[0.02826667 0.50269556 0.3732233  ... 0.56165004 0.47912255 0.26313388]


In [110]:
for i in range (0, len(preds)):
    if i <= 0.5:       
        preds[i] = 1 
    else:  
        preds[i]=0

In [111]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(preds,y_test)
print(accuracy)

0.5908590308370044
